# [MLP Regression](https://github.com/kokchun/Deep-learning-AI21/blob/main/Lectures/Lec0-MLP_regression.ipynb)

In [ ]:
import seaborn
import pandas
import matplotlib.pyplot as plt
import numpy

mpg_df = seaborn.load_dataset('mpg').drop('name', axis=1)
mpg_df.head()

In [ ]:
mpg_df['origin'].value_counts()

In [ ]:
mpg_df.info()

In [ ]:
mpg_df.query('horsepower.isna()') # Exercise: impute the values

In [ ]:
mpg_dropna = mpg_df.dropna(axis=0)
mpg_dropna.info()

In [ ]:
mpg_df['model_year'].value_counts().sort_index().plot(kind='bar', title='Model year counts');

In [ ]:
bins = pandas.IntervalIndex.from_tuples([(69,73), (74,77), (78,82)])
pandas.cut(mpg_dropna['model_year'], bins=bins).head()

In [ ]:
mpg_new_model_year = mpg_dropna.drop(columns='model_year').join(pandas.cut(mpg_dropna['model_year'], bins=bins))
mpg_new_model_year

In [ ]:
mpg_dummies = pandas.get_dummies(mpg_new_model_year, columns=['model_year', 'origin'], drop_first=True)
mpg_dummies.head()

## Train|Test Split

In [ ]:
X, y = mpg_dummies.drop(columns=['mpg']).values, mpg_dummies['mpg'].values # Keras and tensorflow needs arrays instead of dataframes

X, X.shape, type(y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

scaled_X_train.shape, scaled_X_test.shape

## Multiple linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

model_LinearRegression = LinearRegression()
model_LinearRegression.fit(scaled_X_train, y_train)

intercept, coefficients = model_LinearRegression.intercept_, model_LinearRegression.coef_

intercept, coefficients

## Artificial neural network (Shallow Multilinear Perception) (Shallow MLP)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import SGD

model_shallow = Sequential(name='shallow_network') # names cannot have spaces
model_shallow.add(InputLayer(X_train.shape[1])) # 274 features
# model_shallow.add(Dense(20, name='hidden_layer')) # create more hidden layers
model_shallow.add(Dense(1, name='output_layer')) # note no activation function --> linear activation (linear regression)
model_shallow.compile(loss='mean_squared_error', optimizer=SGD(learning_rate=.01))
model_shallow.summary()

In [ ]:
model_shallow.fit(scaled_X_train, y_train, epochs=50, verbose=1, validation_data=(scaled_X_test,y_test))

In [ ]:
model_shallow.history.history.keys()

In [ ]:
loss_df = pandas.DataFrame(model_shallow.history.history)
loss_df.head()

In [ ]:
loss_df.index = range(1, len(loss_df)+1)
loss_df.head()

In [ ]:
loss_df.plot(xlabel='Epochs', ylabel='MSE Loss');

In [ ]:
model_shallow.layers[0]

In [ ]:
weights, bias = model_shallow.layers[0].get_weights()

print(f'Linear regression sklearn: {intercept=}, {coefficients=}')
print(f'ANN: {bias=}, {weights=}')

## Prediction and evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred_ANN = model_shallow.predict(scaled_X_test)
y_pred_LinearRegression = model_LinearRegression.predict(scaled_X_test)

print('MAE, RSME for ANN')
print(mean_absolute_error(y_test, y_pred_ANN), numpy.sqrt(mean_absolute_error(y_test, y_pred_ANN)))
print('\nMAE, RSME for Linear regression')
print(mean_absolute_error(y_test, y_pred_LinearRegression), numpy.sqrt(mean_absolute_error(y_test, y_pred_LinearRegression)))